In [ ]:
import pandas as pd
from bokeh.io import output_notebook
import networkx as nx
from bokeh.plotting import figure, from_networkx, show
from bokeh.palettes import Category20_18



output_notebook()

In [ ]:
df = pd.read_csv('D13_graph4.csv')
df

In [ ]:
df[['ID1', 'ID2','weight' ]] = df['ID1;ID2;weight'].str.split(';', expand=True)
df.drop(columns='ID1;ID2;weight', axis=1, inplace= True)
df = df.astype('int32')
df

In [ ]:
from bokeh.models import (BoxSelectTool, Circle, EdgesAndLinkedNodes,NodesAndLinkedEdges,
                          HoverTool, MultiLine, Plot, Range1d, TapTool)
from bokeh.palettes import Spectral4

G = nx.from_pandas_edgelist(df=df,source='ID1', target='ID2', edge_attr=['weight'])


p = Plot(width= 700, height = 700, title = 'Nodes are clickable and you can hover over edges and nodes')
p.grid.grid_line_color = None

graph = from_networkx(G, nx.circular_layout, scale=1.8, center=(0,0))

p.add_tools(TapTool(),BoxSelectTool())
p.add_tools(HoverTool(renderers=[graph.edge_renderer], tooltips=[('Node 1', '@start'), ('Node 2', '@end'), ('Weight', '@weight')]))
p.add_tools(HoverTool(renderers=[graph.node_renderer], tooltips=[('Node', '@index')]))

graph.node_renderer.glyph = Circle(size=20, fill_color=Spectral4[0])
graph.node_renderer.selection_glyph = Circle(size=15, fill_color=Spectral4[2])
graph.node_renderer.hover_glyph = Circle(size=15, fill_color=Spectral4[1])
graph.node_renderer.data_source.data = dict(
    index = list(range(len(G)))
)

graph.edge_renderer.glyph = MultiLine(line_color="#808080", line_alpha=0.5, line_width=2)
graph.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=2)
graph.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=2)


graph.selection_policy = NodesAndLinkedEdges()
graph.inspection_policy = EdgesAndLinkedNodes()

graph.node_renderer.data_source.data['index'] = list(range(1,len(G)+1))

p.renderers.append(graph)

show(p)

